In [1]:
import pandas as pd
from pathlib import Path

df = pd.read_csv("006_crawling_juznevesti.csv")
df.head()

def exists(yt_hash):
    audiodir = Path("audio")

    file = audiodir / (yt_hash + ".wav")
    return file.exists()

df["exists"] = df.yt_hash.apply(exists)


In [2]:
df.exists.sum()

299

In [3]:
hashes_to_download = df[~df.exists].yt_hash
hashes_to_download

169    VTS0goylW_E
Name: yt_hash, dtype: object

In [4]:
from tqdm.auto import tqdm
import os
import tempfile
from pathlib import Path

audiodir = Path("audio")
def download_sound(yt_hash):
    with tempfile.NamedTemporaryFile() as f:
        cmd = f"youtube-dl {yt_hash} -x --audio-format wav -o {f.name}.wav"
        os.system(cmd)

        cmd = f"ffmpeg -i {f.name}.wav -ac 1 -ar 16000 audio/{yt_hash}.wav"
        os.system(cmd)

from concurrent.futures import ProcessPoolExecutor
with ProcessPoolExecutor(max_workers=4) as executor:
    executor.map(download_sound, hashes_to_download)

[youtube] VTS0goylW_E: Downloading webpage


ERROR: Private video
Sign in if you've been granted access to this video
ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libthe

In [5]:
df.head()

,url,yt_hash,transcript,exists
0,https://www.juznevesti.com/15-minuta/Boban-Sto...,00DtyZ2sCUM,"Gost: Boban Stojanović, redovni profesor Ekono...",True
1,https://www.juznevesti.com/15-minuta/Slavoljub...,58xZSVbpgkk,"Gost: Slavoljub Vlajković, narodni poslanik SP...",True
2,https://www.juznevesti.com/15-minuta/Zoran-Ziv...,YKAqDCyINz4,"Gost: Zoran Žuvković, predsednik Nove stranke\...",True
3,https://www.juznevesti.com/15-minuta/Majkl-Dev...,2QShGnPuVjQ,"Gost: Majkl Devenport, Šef Delegacije EU u Srb...",True
4,https://www.juznevesti.com/15-minuta/Jelena-Dj...,khEIELaXKwI,"Gost: Jelena Đorđević, Odbornica u Skupštini g...",True
